In [1]:
import json
import pandas as pd
import geopandas as gpd

gdf_2010 = gpd.read_file('/Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/landvalue_2010_new.geojson')
#gdf_2010 = gdf_2010.rename(columns={'roll_totalvalue': 'roll_totalvalue_2010'})
#gdf_2010.set_index('ain',inplace=True)
gdf_2010

DriverError: /Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/landvalue_2010_new.geojson: No such file or directory

In [ ]:
n_name = gdf_2010[gdf_2010['CT20'] == 224320]
n_name

In [ ]:
gdf_2010.info()

In [ ]:

gdf_2010.query('roll_landbaseyear == "2010"').roll_landbaseyear.count()

In [ ]:
gdf_2010_sold = gdf_2010[gdf_2010["roll_landbaseyear"] == '2010']
gdf_2010_sold

In [ ]:
min_number = min(gdf_2010_sold.ain)
print(min_number)
max_number = max(gdf_2010_sold.ain)
print(max_number)

In [ ]:
#import parcel data
import requests
import json


url = 'https://data.lacounty.gov/resource/rjze-ndj2.json?$where=within_box(the_geom, 34.1164, -118.3217, 34.01333, -118.2675)&$limit=100000' # copied and pasted from the webpage
r = requests.get(url)
parcel = pd.DataFrame(json.loads(r.text))
parcel

#south:34.013339055368384, -118.23277272855023
#north: 34.11633386937314, -118.2674474941663
#eaast:34.01604254848126, -118.2217864006906
#west: 34.05453104173829, -118.32179797239648
#https://data.cityofchicago.org/resource/yama-9had.json?$where=within_polygon(the_geom, 'MULTIPOLYGON (((34.1037 -118.2484, 34.0054 -118.2455, 34.0545 -118.2062, 34.0633 -118.3351)))')
#within_polygon(location, 'MULTIPOLYGON (((34.10378510012116, -118.24846458486526, 34.00549184389096, -118.24552614565691, 34.05450056185422, -118.20622452124513, 34.06332521000708, -118.33514854151177)))')
#$where=ain>5056000000.0&$limit=2500000
#requestString = 'https://data.lacounty.gov/resource/rjze-ndj2.json&$limit=2100000'
#r = requests.get(requestString)
#d = json.loads(r.text)
#parcel = pd.DataFrame(d)
#parcel

In [ ]:
parcel.info()

In [ ]:
parcelgdf = gpd.GeoDataFrame(
    parcel, geometry=gpd.points_from_xy(parcel.lon_coord, parcel.lat_coord, 
                                          crs='EPSG:4326'))
parcelgdf

In [ ]:
parcelgdf.info()

In [ ]:
gdf_2010_sold.info()

In [ ]:
gdf_2010_sold = gdf_2010_sold.drop(columns=['index_right'])
gdf_2010_sold

In [ ]:
joinedgdf2010=gpd.sjoin(gdf_2010_sold.to_crs('EPSG:4326'),parcelgdf.to_crs('EPSG:4326'),how='left')
joinedgdf2010

In [ ]:
gdf_2010_sold.head()

In [ ]:
join2010 = gdf_2010_sold.merge(parcelgdf, how='left',on='ain')
join2010 = join2010.dropna()
join2010

In [ ]:
join2010.query('neighborhood == "Westlake"').neighborhood.count()

In [ ]:
join2010.info()

In [ ]:
#join2010['roll_landvalue'].astype(float)
#join2010['shape_area'].astype(float)
join2010['shape_area'] = pd.to_numeric(join2010['shape_area'])
join2010['roll_landvalue'] = pd.to_numeric(join2010['roll_landvalue'])

In [ ]:
join2010.info()

In [ ]:
join2010['landvalue_sqft']=join2010['roll_landvalue']/join2010['shape_area']
join2010

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
join2010

In [ ]:

censusjoin2010 = join2010.groupby('CT20').ain.count()
censusjoin2010

In [ ]:
df_census_n = pd.DataFrame(censusjoin2010)
df_census_n =df_census_n.rename(columns={'ain': 'n_parcel'})
df_census_n

In [ ]:
censuslvsfjoin2010 = join2010.groupby('CT20').landvalue_sqft.sum()
censuslvsfjoin2010

In [ ]:
df_lvsf = pd.DataFrame(censuslvsfjoin2010)
df_lvsf

In [ ]:
df_lvsf.info()

In [ ]:
df_census_n.info()

In [ ]:
newjoin = df_lvsf.merge(df_census_n, how='left',on='CT20')
newjoin

In [ ]:
newjoin['avgprice_tract']=newjoin['landvalue_sqft']/newjoin['n_parcel']
newjoin

In [ ]:
#percent change with and without inflation

In [ ]:
avgprice_tract_2020 = pd.read_csv('/Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/avgprice_tract_2020.csv')

In [ ]:
avgprice_tract_2020.set_index('CT20')

In [ ]:
#rename columns
avgprice_tract_2020 = avgprice_tract_2020.rename(columns={'landvalue_sqft': 'landvalue_sqft_2020',
                                                         'n_parcel': 'n_parcel_2020',
                                                         'avgprice_tract':'avgprice_tract_2020'})
avgprice_tract_2020

In [ ]:
#rename columns
avgprice_tract_2010 = newjoin.rename(columns={'landvalue_sqft': 'landvalue_sqft_2010',
                                                         'n_parcel': 'n_parcel_2010',
                                                         'avgprice_tract':'avgprice_tract_2010'})
avgprice_tract_2010

In [ ]:
avgprice_tract_2010['2010avgprice_adjust_to_2020']=avgprice_tract_2010['avgprice_tract_2010']*1.19
avgprice_tract_2010

In [ ]:
avgprice_tract_2010.reset_index(inplace=True)
avgprice_tract_2010.info()

In [ ]:
avgprice_tract_2020.info()

In [ ]:
#2010 to float
avgprice_tract_2010['CT20'] = avgprice_tract_2010.CT20.astype(int)
avgprice_tract_2010['n_parcel_2010'] = avgprice_tract_2010.n_parcel_2010.astype(float)
avgprice_tract_2010.info()

In [ ]:
#2020 to float
avgprice_tract_2020['CT20'] = avgprice_tract_2020.CT20.astype(int)
avgprice_tract_2020['n_parcel_2020'] = avgprice_tract_2020.n_parcel_2020.astype(float)
avgprice_tract_2020.info()

In [ ]:
#join
finaljoin=avgprice_tract_2010.merge(avgprice_tract_2020, how='left',on='CT20')
finaljoin=finaljoin.dropna()
finaljoin

In [ ]:
finaljoin['pct_change_wo_inflation'] = (finaljoin.loc[1:80,'avgprice_tract_2020'] - finaljoin.loc[1:80,'avgprice_tract_2010'])/finaljoin.loc[1:80,'avgprice_tract_2010']
#pct_change_wo_inflation
finaljoin['pct_change_w_inflation']=(finaljoin.loc[1:80,'avgprice_tract_2020'] - finaljoin.loc[1:80,'2010avgprice_adjust_to_2020'])/finaljoin.loc[1:80,'2010avgprice_adjust_to_2020']
#pct_change_w_inflation
finaljoin

In [ ]:
#finaljoin['CT20']=finaljoin.CT20.astype(int)
#gdf_2010['CT20']=gdf_2010.CT20.astype(int)
#finaljoin.info()

In [ ]:
#gdf_2010.set_index('CT20',inplace=True)
#finaljoin.set_index('CT20',inplace=True)

In [ ]:
#finaljoin2=finaljoin.join(gdf_2010.drop(columns='CT20'))
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#finaljoin2

In [ ]:
census = gpd.read_file('/Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/Data/Census_Tracts_2020.geojson')
census.head()

In [ ]:
gdf_2010.info()

In [ ]:
census.info()

In [ ]:
census['CT20']=census.CT20.astype(int)

In [ ]:
#use this for mapping
census_finaljoin=finaljoin.merge(census, on='CT20',how='left')
census_finaljoin


In [ ]:
#neighborhood boundary
nb= gpd.read_file('/Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/Data/LA_Times_Neighborhood_Boundaries (1)-Copy1.geojson')
#nb.name
wl=nb[nb.name=='Westlake']
kt=nb[nb.name=='Koreatown']
pu=nb[nb.name=='Pico-Union']
sl=nb[nb.name=='Silver Lake']
dt=nb[nb.name=='Downtown']
ep=nb[nb.name=='Echo Park']
join1 = pd.concat([wl, kt], axis=0).reset_index(drop=True)
join2 = pd.concat([join1, pu], axis=0).reset_index(drop=True)
join3 = pd.concat([join2, sl], axis=0).reset_index(drop=True)
join4 = pd.concat([join3, dt], axis=0).reset_index(drop=True)
nb = pd.concat([join4, ep], axis=0).reset_index(drop=True)
nb



In [ ]:
census_finaljoin.CT20=census_finaljoin.CT20.astype(object)

In [ ]:
#merge census final join with nb
#no good
mergetest=census_finaljoin.merge(gdf_2010, on='CT20')
mergetest

In [ ]:
import geopandas as gpd
gdf=gpd.GeoDataFrame(census_finaljoin,geometry='geometry')

In [ ]:
gdf_2010.info()

In [ ]:
gdf_2010= gdf_2010.drop(columns=['index_right'])
gdf_2010

In [ ]:
mergetest2=gpd.sjoin(gdf.to_crs('EPSG:4326'),gdf_2010.to_crs('EPSG:4326'),how='left')
mergetest2

In [ ]:
census_finaljoin.info()

In [ ]:
gdf.columns

In [ ]:
#change to geodataframe
import geopandas as gpd
gdf=gpd.GeoDataFrame(census_finaljoin,geometry='geometry')

In [ ]:
#map
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
gdf.to_crs('EPSG:3857').plot('pct_change_wo_inflation',
                             markersize=0.001, 
                             ax=ax, legend=True)
#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_without_inflation', fontsize=20 )

In [ ]:
gdf.pct_change_w_inflation.max()

In [ ]:
gdf.pct_change_w_inflation.min()

In [ ]:
gdf.pct_change_w_inflation.median()

In [ ]:
#map
#natural break
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
#gdf.to_crs('EPSG:3857').plot('pct_change_w_inflation',
                             #markersize=0.001, 
                             #ax=ax, legend=True)
        
gdf.to_crs('EPSG:3857').plot(figsize=(12,12),
            column='pct_change_w_inflation', 
            cmap='RdYlGn_r', 
            scheme='NaturalBreaks', # to use custom breaks
            #classification_kwds={'bins':[-1,50,]}, # define the breaks here
            edgecolor='white', 
            linewidth=0.1, 
            legend=True,
            ax=ax
           )

#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_with_inflation', fontsize=20 )

In [ ]:
#equal interval
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
#gdf.to_crs('EPSG:3857').plot('pct_change_w_inflation',
                             #markersize=0.001, 
                             #ax=ax, legend=True)
        
gdf.to_crs('EPSG:3857').plot(figsize=(12,12),
            column='pct_change_w_inflation', 
            cmap='RdYlGn_r', 
            scheme='equal_interval', # to use custom breaks
            #classification_kwds={'bins':[-1,50,]}, # define the breaks here
            edgecolor='white', 
            linewidth=0.1, 
            legend=True,
            ax=ax
           )

#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_with_inflation', fontsize=20 )

In [ ]:
#quantiles
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
#gdf.to_crs('EPSG:3857').plot('pct_change_w_inflation',
                             #markersize=0.001, 
                             #ax=ax, legend=True)
        
gdf.to_crs('EPSG:3857').plot(figsize=(12,12),
            column='pct_change_w_inflation', 
            cmap='RdYlGn_r', 
            scheme='quantiles', # to use custom breaks
            #classification_kwds={'bins':[-1,50,]}, # define the breaks here
            edgecolor='white', 
            linewidth=0.1, 
            legend=True,
            ax=ax
           )

#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_with_inflation', fontsize=20 )

In [ ]:
#map
#natural break
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
#gdf.to_crs('EPSG:3857').plot('pct_change_w_inflation',
                             #markersize=0.001, 
                             #ax=ax, legend=True)
        
gdf.to_crs('EPSG:3857').plot(figsize=(12,12),
            column='pct_change_w_inflation', 
            cmap='RdYlGn_r', 
            scheme='user_defined', # to use custom breaks
            classification_kwds={'bins':[-1,-0.5,0,0.5,1,2,5,10,20,30,40,50]}, # define the breaks here
            edgecolor='white', 
            linewidth=0.1, 
            legend=True,
            ax=ax
           )

#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_with_inflation', fontsize=20 )

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

xa = np.linspace(0, 5, 20)

fig, ax = plt.subplots(figsize=(10,10))
#gdf.to_crs('EPSG:3857').plot('pct_change_w_inflation',
                             #markersize=0.001, 
                             #ax=ax, legend=True)
        
gdf.to_crs('EPSG:3857').plot(figsize=(12,12),
            column='pct_change_w_inflation', 
            cmap='RdYlGn_r', 
            scheme='NaturalBreaks', # to use custom breaks
            #classification_kwds={'bins':[-1,50,]}, # define the breaks here
            edgecolor='white', 
            linewidth=0.1, 
            legend=True,
            ax=ax
           )
nb=xa**2
nb.to_crs('EPSG:3857').plot('name',
                           color='lightcoral', linewidth=4, linestyle=':', 
                            ax=ax)

#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('land value pct_change_with_inflation', fontsize=20 )

In [ ]:
m = folium.Map(zoom_start=10,
              location=[34.052235, -118.243683])
folium.Choropleth(
    geo_data='/Users/pearlliu100/Documents/GitHub/UDS-Project--Full-Team-/landvalue_pctchange.geojson',
    name='pct_change_w_inflation',
    data=gdf,
    popup=gdf.pct_change_w_inflation,
    columns=['CT20', 'pct_change_w_inflation'],
    fill_color='YlGn',
    key_on='feature.properties.CT20',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='pct_change_w_inflation'
).add_to(m)

m

In [ ]:
for index, row in nb.iterrows():
    # add folium marker code
    folium.Marker([row.LAT, row.LONG], popup=row.name, tooltip=row.STATION).add_to(m)
m

In [ ]:
folium.Choropleth(
    geo_data=gdf,
    name='pct_change_w_inflation',
    data=gdf,
    popup=gdf.pct_change_w_inflation,
    columns=['CT20', 'pct_change_w_inflation'],
    fill_color='YlGn',
    key_on='feature.properties.CT20',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='pct_change_w_inflation'
).add_to(m)

# show the map
m

In [ ]:
m =folium.Map(location=[34.051850, -118.243322], tiles=None) #Center map on LA County

#Adding basemap
folium.raster_layers.TileLayer(
    location=[34.051850, -118.243322],
    tiles='https://api.mapbox.com/styles/v1/jinglan9803/ckvb8q3fa8ocm15s2h2chib9k/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiamluZ2xhbjk4MDMiLCJhIjoiY2wzd2w2eGRhMGhkZjNpcnF5ZWtocHZrciJ9.iJrdUieRYeaerd-cqJLNzw',
    attr='Mapbox',
    API_key='pk.eyJ1IjoiamluZ2xhbjk4MDMiLCJhIjoiY2wzd2wyeDZyMTR2ZDNpcDQ1a3RvN2FnaCJ9.HaceX4lYSqLYsUpUQE9X_Q',
    name='Monochrome',
    zoom_start=9  # Limited levels of zoom for free Mapbox tiles.
).add_to(m)

#Chloropleth
folium.Choropleth(
    geo_data=gdf,
    name='pct_change_w_inflation',
    data=gdf,
    popup=gdf.pct_change_w_inflation,
    columns=['CT20', 'pct_change_w_inflation'],
    fill_color='YlGn',
    key_on='feature.properties.CT20',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='pct_change_w_inflation'
).add_to(m)

#second layer to add to map
f2=folium.FeatureGroup(name='nb').add_to(m)



#m.save('schoolsandstores.html')

m

In [ ]:
import pandas as pd
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

import plotly.express as px

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#markers
#for index, row in gdf_trimmed.iterrows():
    #folium.Circle([row.latitude, row.longitude], popup=row.Name, tooltip=row.Name, color="purple").add_to(f2)

#f3=folium.FeatureGroup(name='Trader Joes Stores').add_to(m)

for index, row in tj_trimmed.iterrows():
    folium.Circle([row.Latitude, row.Longitude], popup=row.Store_Chain, tooltip=row.Store_Chain,color="blue").add_to(f3)

f4=folium.FeatureGroup(name='Whole Foods Stores').add_to(m)

for index, row in wf_trimmed.iterrows():
    folium.Circle([row.Latitude, row.Longitude], popup=row.Store_Chain, tooltip=row.Store_Chain,color="green").add_to(f4)

f5=folium.FeatureGroup(name='Vons Stores').add_to(m)

for index, row in vns_trimmed.iterrows():
    folium.Circle([row.Latitude, row.Longitude], popup=row.Store_Chain, tooltip=row.Store_Chain,color="crimson").add_to(f5)

    
folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(m)

In [ ]:

fig, ax = plt.subplots(figsize=(10,10))
gdf_2010.to_crs('EPSG:3857').plot('neighborhood',
                             #markersize=1, 
                             #ax=ax, legend=True)
#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])

nb.to_crs('EPSG:3857').plot('name',
                           markersize=1, 
                            ax=ax, legend=True)

ax.set_title('Neighborhood by Ain', fontsize=20 )


In [ ]:
#map
import contextily as ctx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20,20))
nb.to_crs('EPSG:3857').plot('name',
                             markersize=0.001, 
                             ax=ax, legend=True)
#ax.set_ylim([3.98e6,4.14e6])
ctx.add_basemap(ax=ax)    
#ax.set_xticks([])
#ax.set_yticks([])
ax.set_title('nb', fontsize=40 )


In [ ]:
gdf_2010_sjoin.to_file('landvalue_2010_new.geojson', driver='GeoJSON')